# Data prepration

In [1]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from functions import mean_gen, varx_gen, vary_gen, spherical_data, discrete_angles

2022-08-22 13:23:24.297659: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-22 13:23:24.297715: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
n = 200000


m_th = 50
m_ph = 50
thetas = discrete_angles(n, m_th, 2)
phis = discrete_angles(n, m_ph, 1)
name = ('discrete_model_mth%.0f_mph%.0f' %(m_th, m_ph))


# angles = np.array([0, 0])
# thetas, phis = angles[0] * np.ones(n), angles[1] * np.ones(n)
# name = ('discrete_model_th%.0f_ph%.0f' %(angles[0], angles[1]))

x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis)

print(x_train)

[[ 0.3554867   0.58188832  6.02824407  2.01001591]
 [-0.52893511  0.76865303  4.96989645  1.7522671 ]
 [-0.14182714  0.44639446  1.96965008  2.25649864]
 ...
 [-0.81233133  0.15858867  4.7699055   2.50942657]
 [ 1.66389881  1.40568226  0.01363396  0.05113504]
 [-0.50813679 -1.64994621  0.00896637  0.58824073]]


# Training Interpolated Model

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
]);

loss_fn = tf.keras.losses.BinaryCrossentropy()
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

batch_size = 5000

checkpoint_path = "tutorial/checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1,
    save_freq=10*batch_size
)

log_dir = "tutorial/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

EPOCHS=30
model.fit(
    x_train, y_train, epochs=EPOCHS,
    batch_size=batch_size,
    callbacks=[cp_callback, tensorboard_callback]
);

model.save('3dmodels/' + name)

2022-08-22 13:23:33.669336: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-22 13:23:33.669371: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-22 13:23:33.669393: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-matan): /proc/driver/nvidia/version does not exist
2022-08-22 13:23:33.669729: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-22 13:23:33.688341: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initiali

Epoch 1/30
 3/64 [>.............................] - ETA: 5s - loss: 0.6981 - accuracy: 0.5047 

2022-08-22 13:23:34.558996: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-08-22 13:23:34.559033: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-08-22 13:23:34.589348: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-08-22 13:23:34.595870: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-08-22 13:23:34.627656: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tutorial/logs/20220822-132333/train/plugins/profile/2022_08_22_13_23_34

2022-08-22 13:23:34.634989: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to tutorial/logs/20220822-132333/train/plugins/profile/2022_08_22_13_23_34/jupyter-matan.trace.json.gz
2022-08-22 13:23:34.653094: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tutorial/logs/20220822-132

64/64 [==============================] - 1s 15ms/step - loss: 0.6604 - accuracy: 0.5909
Epoch 2/30
64/64 [==============================] - 1s 11ms/step - loss: 0.6519 - accuracy: 0.5993
Epoch 3/30
64/64 [==============================] - 1s 11ms/step - loss: 0.6501 - accuracy: 0.6006
Epoch 4/30
64/64 [==============================] - 1s 11ms/step - loss: 0.6489 - accuracy: 0.6018
Epoch 5/30
64/64 [==============================] - 1s 10ms/step - loss: 0.6479 - accuracy: 0.6020
Epoch 6/30
64/64 [==============================] - 1s 11ms/step - loss: 0.6473 - accuracy: 0.6028
Epoch 7/30
64/64 [==============================] - 1s 11ms/step - loss: 0.6467 - accuracy: 0.6042
Epoch 8/30
64/64 [==============================] - 1s 11ms/step - loss: 0.6463 - accuracy: 0.6050
Epoch 9/30
64/64 [==============================] - 1s 11ms/step - loss: 0.6459 - accuracy: 0.6061
Epoch 10/30
64/64 [==============================] - 1s 11ms/step - loss: 0.6455 - accuracy: 0.6073
Epoch 11/30
64/64 [=

2022-08-22 13:23:56.466466: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: 3dmodels/discrete_model_mth50_mph50/assets


In [16]:
model = tf.keras.models.load_model('3dmodels/' + name)